In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
file = '../data/product_merged.csv'
df = pd.read_csv(file, low_memory=False)

In [19]:
import re
special = re.compile(r'http\S+|www\S+|[^a-zA-Z ]+|xx+')
docs = [' '.join(special.sub('', doc.lower()).split()) for doc in df['Consumer complaint narrative'].values]

In [21]:
df['cleaned_text'] = docs

In [24]:
nlp.vocab['$'].is_punct = True

In [25]:
tokenized = []
for doc in nlp.pipe(docs, disable=['tagger', 'parser', 'ner']):
    tokenized.append(" ".join(token.lemma_.lower() for token in doc if not token.is_stop and not token.is_space \
            and not token.is_punct and not token.like_num))

In [26]:
df['tokenized_text'] = tokenized

In [27]:
df.to_csv('../data/with_tokenized.csv', index=False)

In [7]:
df[df['Product'] == 'Debt collection']['Sub-product'].value_counts()

I do not know                            18505
Other debt                               16102
Other (i.e. phone, health club, etc.)    12389
Credit card debt                         12224
Medical debt                              9556
Credit card                               7520
Medical                                   6990
Payday loan debt                          2151
Payday loan                               2064
Auto debt                                 1981
Mortgage debt                             1341
Auto                                      1064
Federal student loan debt                  966
Private student loan debt                  961
Mortgage                                   907
Non-federal student loan                   684
Federal student loan                       549
Name: Sub-product, dtype: int64

In [8]:
dont_knows = df[(df['Product'] == 'Debt collection') & (df['Sub-product'] == 'I do not know')]

In [14]:
dont_knows.iloc[4]['Consumer complaint narrative']

'I called the phone number back that has been calling me daily for weeks now ( XXXX ) only to hear a prerecorded message stating that it is Sequium Asset Solutions attempting to collect a debt. There are no outstanding debts for either myself or my husband, with the exceptions of our home, cars, student loans, and credits cards, all of which are up-to-date on their payments. I believe these phone calls to be an attempt to extort information from me due to my having good to outstanding credit.'